In [1]:
import cv2
import numpy as np
import pandas as pd
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
model = ResNet50(weights='imagenet', include_top=False, pooling='avg')

def preprocess_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.resize(image, (224, 224)) 
    image = preprocess_input(image)  
    return image

In [3]:
def extract_features(image):
    features = model.predict(np.expand_dims(image, axis=0))
    return features.flatten()  

In [ ]:
import pandas as pd
import numpy as np
import os

def add_image_feature(image_path):
    image_name = image_path.split("\\")[-1].split('.')[0]  
    image = preprocess_image(image_path)
    features = extract_features(image)
    feature_str = ",".join(map(str, features))
    file_exists = os.path.isfile('feature_data.csv')
    new_data = pd.DataFrame([[image_name, feature_str]], columns=["filename", "features"])
    new_data.to_csv('feature_data.csv', mode='a', index=False, header=not file_exists)

    print(f"Added {image_name} to {'feature_data.csv'}")

add_image_feature('data\A_f0027_08.png')


1/1 [==============================] - 0s 121ms/step
Added A_f0027_08 to feature_data.csv


In [7]:
def compute_similarity(features1, features2):
    return cosine_similarity([features1], [features2])[0][0]

In [33]:
import ast
def match_fingerprint(new_fingerprint_path, threshold=0.9):

    new_features = extract_features(preprocess_image(new_fingerprint_path))
    
    best_match = None
    highest_similarity = -1  
    dataf=pd.read_csv("feature_data.csv")
    
    for index, row in dataf.iterrows():
        similarity = compute_similarity(new_features, ast.literal_eval(row['extract_features']))
        if similarity > highest_similarity:
            highest_similarity = similarity
            best_match = row['image_name']
    
    if highest_similarity >= threshold:
        print(f"Match found: {best_match} (Similarity: {highest_similarity})")
        return best_match
    else:
        print("No match found. Fingerprint not in the dataset.")
        return None
match_fingerprint("data\A_f0027_08.png")

1/1 [==============================] - 0s 117ms/step
Match found: A_f0027_08 (Similarity: 0.9999999999999993)


'A_f0027_08'